In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import os
import json

In [3]:
def BEA(method, **kwargs):
    '''
    methods: getdatasetlist, getparameterlist, getparametervalues, getparametervaluesfiltered
    params: datasetname, parametername, targetparameter, tablename, linecode, year, geoflips
    '''
    target = 'https://apps.bea.gov/api/data'
    payload = {'UserID': os.environ['BEA_API_KEY'], 'method':method}
    payload.update(kwargs)
    return requests.get(target, params=payload)

In [4]:
def clean_BEA(df,labelname):
    df=df.drop(labels=['Code','CL_UNIT','NoteRef','UNIT_MULT','TimePeriod','GeoName'],axis=1)
    df=df.astype({'GeoFips':'float64'})
    df=df[df['GeoFips']%2==1]
    df=df.rename(columns={'DataValue':labelname})
    return df

In [5]:
vote=pd.read_csv('CAPSTONE_1_files/countypres_2000-2016.csv')
vote.rename(columns = {'FIPS':'GeoFips'}, inplace = True)
vote=vote[vote['year']==2016]

In [6]:
#df=pd.DataFrame.from_records(resp.json()['BEAAPI']['Results']['Data'])
#method 1

In [7]:
#df1=pd.read_json(json.dumps(resp.json()['BEAAPI']['Results']['Data']))
#method 2

In [8]:
#df=pd.DataFrame.from_dict(resp.json()['BEAAPI']['Results']['Data'])
#method 3

In [10]:
resp=BEA('getdata', datasetname='regional',geofips='County',tablename='CAEMP25N',year='2016',linecode=10)
#num jobs
df=pd.DataFrame.from_records(resp.json()['BEAAPI']['Results']['Data'])
df=clean_BEA(df,'num_jobs')
merged=pd.merge(vote,df,how='left',on='GeoFips')

In [11]:
resp=BEA('getdata', datasetname='regional',geofips='County',tablename='CAINC30',linecode=10)
#personal income total
df=pd.DataFrame.from_records(resp.json()['BEAAPI']['Results']['Data'])
df=clean_BEA(df,'total_personal_income_thous')
merged=pd.merge(merged,df,how='left',on='GeoFips')

In [12]:
resp=BEA('getdata', datasetname='regional',geofips='County',tablename='CAINC45',year='2016',linecode=370)
#farm income thousands
df=pd.DataFrame.from_dict(resp.json()['BEAAPI']['Results']['Data'])
df=clean_BEA(df,'Farm_income_thousands')
merged=pd.merge(merged,df,how='left',on='GeoFips')

In [13]:
resp=BEA('getdata', datasetname='regional',geofips='County',tablename='CAINC30',year='2016',linecode=110)
#income/capita
df=pd.DataFrame.from_dict(resp.json()['BEAAPI']['Results']['Data'])
df=clean_BEA(df,'Income/Capita')
merged=pd.merge(merged,df,how='left',on='GeoFips')

In [14]:
resp=BEA('getdata', datasetname='regional',geofips='County',tablename='CAINC30',year='2016',linecode=100)
#population
df=pd.DataFrame.from_dict(resp.json()['BEAAPI']['Results']['Data'])
df=clean_BEA(df,'population')
merged=pd.merge(merged,df,how='left',on='GeoFips')

In [15]:
resp=BEA('getdata', datasetname='regional',geofips='County',tablename='CAINC30',year='2016',linecode=290)
#average salary
df=pd.DataFrame.from_dict(resp.json()['BEAAPI']['Results']['Data'])
df=clean_BEA(df,'avg_sal')
merged=pd.merge(merged,df,how='left',on='GeoFips')

In [16]:
df=pd.read_excel('CAPSTONE_1_files/laucnty16.xlsx', converters={'state_fips_Code':str, 'county_fips_code':str})
df['GeoFips']=df['state_fips_Code']+df['county_fips_code']
df=df.astype({'GeoFips':'float64'})
df=df.drop(labels=['LAUS_Code','state_fips_Code','county_fips_code','Unnamed: 5', 'Year','County Name/State Abbreviation'], axis=1)
merged=pd.merge(merged,df,how='left',on='GeoFips')

In [17]:
df=merged
df['share_vote']=df['candidatevotes']/df['totalvotes']
df.head()

,year,state,state_po,county,GeoFips,office,candidate,party,candidatevotes,totalvotes,...,total_personal_income_thous,Farm_income_thousands,Income/Capita,population,avg_sal,labor_Force,Employed,Unemployed,Unemployment_Rate,share_vote
0,2016,Alabama,AL,Autauga,1001.0,President,Hillary Clinton,democrat,5936.0,24973,...,"2,116,276","3,073","39,509","55,278","36,966",25966.0,24645.0,1321.0,5.1,0.237697
1,2016,Alabama,AL,Autauga,1001.0,President,Hillary Clinton,democrat,5936.0,24973,...,"2,183,987","3,073","39,509","55,278","36,966",25966.0,24645.0,1321.0,5.1,0.237697
2,2016,Alabama,AL,Autauga,1001.0,President,Hillary Clinton,democrat,5936.0,24973,...,"2,247,046","3,073","39,509","55,278","36,966",25966.0,24645.0,1321.0,5.1,0.237697
3,2016,Alabama,AL,Autauga,1001.0,President,Donald Trump,republican,18172.0,24973,...,"2,116,276","3,073","39,509","55,278","36,966",25966.0,24645.0,1321.0,5.1,0.727666
4,2016,Alabama,AL,Autauga,1001.0,President,Donald Trump,republican,18172.0,24973,...,"2,183,987","3,073","39,509","55,278","36,966",25966.0,24645.0,1321.0,5.1,0.727666
